In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil
import networkx as nx



In [2]:
# read the network topology that is HDMA hydrolakes global created by the repository
ds = xr.open_dataset('/Users/shg096/Desktop/mizuRoute_HDMA_Global/data/Network_topology_HDMA_HydroLake_v1.nc')
ds

<xarray.Dataset>
Dimensions:    (n: 298277)
Coordinates:
    ID         (n) float64 ...
Dimensions without coordinates: n
Data variables:
    ID_next    (n) float64 ...
    length     (n) float64 ...
    area       (n) float64 ...
    lake       (n) float64 ...
    endorheic  (n) float64 ...
    slope      (n) float64 ...
    S_0        (n) float64 ...
    S_max      (n) float64 ...
    Coeff      (n) float64 ...
    power      (n) float64 ...
    lake_type  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:50:18 2023
    source:       Written by test script of utilities (https://github.com/She...

In [3]:
target_ID = 3044094 # segment ID downstream of lake victoria

# convert ds back to df
df = ds.to_dataframe()

# drop columns from df is available
df = df.drop(['target_vol','lake_type_for_target_vol'], axis=1, errors='ignore')

# make the string out of ID and then slice the dataframe based on 2 (africa) and 7 lakes
df ['ID_str'] = df['ID'].astype(str)
df = df[df['ID_str'].astype(str).str.startswith(('3', '7'))]

# create the riv graph
riv_graph = nx.from_pandas_edgelist(df,\
                                    source='ID',\
                                    target='ID_next',\
                                    create_using=nx.DiGraph)
nodes = nx.ancestors(riv_graph, target_ID)
nodes = np.array(list(nodes))
nodes = np.append(nodes, target_ID)

# now slice the ds based on existing nodes
idx = np.where(np.isin(ds['ID'].values, nodes))
idx = np.array(idx).flatten()
ds_slice = ds.isel(n=idx)

# find the location of target segment and add -9999 as downstream
idx = np.where(np.isin(ds_slice['ID'].values, target_ID))
idx = np.array(idx).flatten()
ds_slice ['ID_next'][idx] = -9999

ds_slice


<xarray.Dataset>
Dimensions:    (n: 290)
Coordinates:
    ID         (n) float64 3e+06 3e+06 3e+06 ... 7.008e+06 7.008e+06 7.1e+06
Dimensions without coordinates: n
Data variables:
    ID_next    (n) float64 3.022e+06 7.001e+06 3.033e+06 ... 3.035e+06 3.026e+06
    length     (n) float64 347.9 1.0 2.229e+03 4.591e+03 ... 1.0 1.0 1.0 1.0
    area       (n) float64 2.486e+08 2.444e+08 2.512e+08 ... 2.163e+07 4.41e+06
    lake       (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    endorheic  (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    slope      (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    S_0        (n) float64 1e+06 1e+06 1e+06 ... 3.261e+08 4.9e+08 1.829e+07
    S_max      (n) float64 1.244e+09 1.223e+09 1.257e+09 ... 5.982e+08 4.034e+07
    Coeff      (n) float64 0.01 0.01 0.01 0.01 0.01 ... 0.01 0.01 0.01 0.01 0.01
    power      (n) float64 1.5 1.5 1.5 1.5 1.5 1.5 ... 1.5 1.5 1.5 1.5 1.5 1.5
    lake_type  (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:50:18 2023
    source:       Written by test script of utilities (https://github.com/She...

In [4]:
ds_merit_lake = xr.open_dataset('../data/network_topology/Network_topology_merit_lake.nc')
df_merit_lake = ds_merit_lake.to_dataframe()

# Define a custom function to add 7000000 to values smaller than 10000000
def add_7000000(value):
    if pd.notna(value) and value < 70000000:
        return value + 7000000
    return value

# Apply the custom function to the 'ID' column
df_merit_lake['ID'] = df_merit_lake['ID'].apply(add_7000000)
df_merit_lake['ID'] = df_merit_lake['ID'].astype(int)

# slice to only lake values
df_merit_lake = df_merit_lake[df_merit_lake['is_lake']==1]
filtered_columns = df_merit_lake.columns[df_merit_lake.columns.str.startswith(('HYPE_', 'D03_', 'H06_')) \
                                         | (df_merit_lake.columns == 'ID')\
                                         | (df_merit_lake.columns == 'lake_type_Doll')\
                                         | (df_merit_lake.columns == 'lake_type_Doll_HYPE')\
                                         | (df_merit_lake.columns == 'lake_type_Doll_HYPE_HANASAKI')]
df_merit_lake = df_merit_lake[filtered_columns]
df_merit_lake


# getting the lake param and appending it to the HDMA segmenets
df_HDMA_lake = ds_slice.to_dataframe()
df_HDMA_lake ['ID'] = df_HDMA_lake ['ID'].astype(int)
lake_param = pd.merge(df_HDMA_lake, df_merit_lake, on=['ID'], how='inner')
df_HDMA_lake_segs = df_HDMA_lake [df_HDMA_lake['lake']==0]
Ntopo = pd.concat([lake_param, df_HDMA_lake_segs], ignore_index=True)
Ntopo.to_csv('../data/network_topology/Network_topology_HDMA_with_lake.csv')


In [7]:
# save the network topology
df = pd.read_csv('../data/network_topology/Network_topology_HDMA_with_lake.csv')

# prepare and rename some variables for Doll 2003
df['lake_vol'] = df['D03_S_0'] # create the new variable
df['is_lake']  = df['lake'] # create the new variable
# rename to more understandbale variables
new_names_Doll = {'D03_S_0': 'D03_S_0',
                  'Doll_Coef': 'D03_Coef',
                  'Doll_Power': 'D03_Power',
                  'Doll_MaxStorage': 'D03_S_max'}
# define the variable name, long name and units
var_info_Doll = {'D03_S_0':   {'long_name': 'lake inactive volume',  'unit': 'm**3'},
                 'D03_Coef':  {'long_name': 'Doll coefficient often taken as 0.01 per day', 'unit': 'd*-1'},
                 'D03_Power': {'long_name': 'Doll power',  'unit': '-'},
                 'D03_S_max': {'long_name': 'lake maximume storage, often 5 meter above inactive storage', 'unit': 'm**3'}}
# define the saving mechanism
var_encoding_Doll = {'D03_S_0':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_Coef':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_Power': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_S_max': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9}}
#
new_names_Doll_values = list(new_names_Doll.values())
var_info_Doll_keys = list(var_info_Doll.keys())
var_encoding_Doll_keys = list(var_encoding_Doll.keys())
if not new_names_Doll_values == var_info_Doll_keys == var_encoding_Doll_keys:
    sys.exit('keys and values are not similar')


# prepare and rename varibales for Hanasaki 2006
new_names_Hanasaki = {'H06_alpha': 'H06_alpha',
                      'H06_envfact': 'H06_envfact',
                      'H06_c1': 'H06_c1',
                      'H06_c2':'H06_c2',
                      'H06_exponent':'H06_exponent',
                      'H06_denominator': 'H06_denominator',
                      'H06_c_compare': 'H06_c_compare',
                      'H06_frac_Sdead': 'H06_frac_Sdead',
                      'H06_I_mem_F': 'H06_I_mem_F',
                      'H06_D_mem_F': 'H06_D_mem_F',
                      'H06_I_mem_L': 'H06_I_mem_L',
                      'H06_D_mem_L': 'H06_D_mem_L',
                      'H06_E_rel': 'H06_E_rel_init',
                      'H06_purpose': 'H06_purpose',
                      'inflow_Jan': 'H06_I_Jan',
                      'inflow_Feb': 'H06_I_Feb',
                      'inflow_Mar': 'H06_I_Mar',
                      'inflow_Apr': 'H06_I_Apr',
                      'inflow_May': 'H06_I_May',
                      'inflow_Jun': 'H06_I_Jun',
                      'inflow_Jul': 'H06_I_Jul',
                      'inflow_Aug': 'H06_I_Aug',
                      'inflow_Sep': 'H06_I_Sep',
                      'inflow_Oct': 'H06_I_Oct',
                      'inflow_Nov': 'H06_I_Nov',
                      'inflow_Dec': 'H06_I_Dec',
                      'H06_D_Jan': 'H06_D_Jan',
                      'H06_D_Feb': 'H06_D_Feb',
                      'H06_D_Mar': 'H06_D_Mar',
                      'H06_D_Apr': 'H06_D_Apr',
                      'H06_D_May': 'H06_D_May',
                      'H06_D_Jun': 'H06_D_Jun',
                      'H06_D_Jul': 'H06_D_Jul',
                      'H06_D_Aug': 'H06_D_Aug',
                      'H06_D_Sep': 'H06_D_Sep',
                      'H06_D_Oct': 'H06_D_Oct',
                      'H06_D_Nov': 'H06_D_Nov',
                      'H06_D_Dec': 'H06_D_Dec'}
# define the variable name, long name and units
var_info_Hanasaki =  {'H06_alpha':       {'long_name': 'fraction of active storage compared to the total storage',  'unit': '-'},
                      'H06_envfact':     {'long_name': 'fraction of yearly mean inflow that can be used to meet demands',  'unit': '-'},
                      'H06_c1':          {'long_name': 'first coefficient of target release calculation',  'unit': '-'},
                      'H06_c2':          {'long_name': 'second coefficient of target release calculation',  'unit': '-'},
                      'H06_exponent':    {'long_name': 'exponent in actual release calculation',  'unit': 'm**3'},
                      'H06_denominator': {'long_name': 'denominator in actual release calculation',  'unit': 'm**3'},
                      'H06_c_compare':   {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                      'H06_frac_Sdead':  {'long_name': 'fraction of maximume storage that define dead storage, 0.1',  'unit': '-'},
                      'H06_I_mem_F':     {'long_name': 'logical parameter to activate memory for inflow',  'unit': '-'},
                      'H06_D_mem_F':     {'long_name': 'logical parameter to activate memory for demand',  'unit': '-'},
                      'H06_I_mem_L':     {'long_name': 'the length of the memory in years for inflow',  'unit': '-'},
                      'H06_D_mem_L':     {'long_name': 'the length of the memory in years for demand',  'unit': '-'},
                      'H06_E_rel_init':  {'long_name': 'release before the hanasaki start year',  'unit': '-'},
                      'H06_purpose':     {'long_name': 'Reservoir purpose (0 non-irrigation, 1 irrigation)',  'unit': '-'},
                      'H06_I_Jan':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Feb':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Mar':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Apr':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_May':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Jun':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Jul':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Aug':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Sep':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Oct':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Nov':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Dec':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jan':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Feb':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Mar':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Apr':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_May':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jun':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jul':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Aug':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Sep':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Oct':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Nov':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Dec':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'}}
# define saving mechanism
var_encoding_Hanasaki ={'H06_alpha':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_envfact':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c1':          {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c2':          {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_exponent':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_denominator': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c_compare':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_frac_Sdead':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_mem_F':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_D_mem_F':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_I_mem_L':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_D_mem_L':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_E_rel_init':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_purpose':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_I_Jan':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Feb':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Mar':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Apr':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_May':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Jun':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Jul':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Aug':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Sep':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Oct':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Nov':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Dec':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jan':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Feb':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Mar':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Apr':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_May':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jun':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jul':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Aug':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Sep':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Oct':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Nov':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Dec':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},}
#
new_names_Hanasaki_values = list(new_names_Hanasaki.values())
var_info_Hanasaki_keys = list(var_info_Hanasaki.keys())
var_encoding_Hanasaki_keys = list(var_encoding_Hanasaki.keys())
if not new_names_Hanasaki_values == var_info_Hanasaki_keys == var_encoding_Hanasaki_keys:
    sys.exit('keys and values are not similar')


# prepare and rename HYPE parameters
new_names_HYPE   = {'EL_SPL':    'HYPE_SPL_EL',
                    'SPL_COEFF': 'HYPE_SPL_COEFF',
                    'SPL_EXP':   'HYPE_SPL_EXP',
                    'EL_LIM':    'HYPE_LIM_EL',
                    'EL_MIN':    'HYPE_MIN_EL',
                    'EL_ZERO':   'HYPE_ZERO_EL',
                    'OPS_AMP':   'HYPE_PRIM_AMP',
                    'OPS_PHA':   'HYPE_PRIM_PHASE',
                    'OPS_MEAN':  'HYPE_PRIM_MEAN',
                    'PRM_FLAG':  'HYPE_PRIM_FLAG'}
# define the varibale name, long name
var_info_HYPE = {'HYPE_SPL_EL':     {'long_name': 'fraction of active storage compared to the total storage',  'unit': '-'},
                 'HYPE_SPL_COEFF':  {'long_name': 'fraction of yearly mean inflow that can be used to meet demands',  'unit': '-'},
                 'HYPE_SPL_EXP':    {'long_name': 'first coefficient of target release calculation',  'unit': '-'},
                 'HYPE_LIM_EL':     {'long_name': 'second coefficient of target release calculation',  'unit': '-'},
                 'HYPE_MIN_EL':     {'long_name': 'exponent in actual release calculation',  'unit': 'm**3'},
                 'HYPE_ZERO_EL':    {'long_name': 'denominator in actual release calculation',  'unit': 'm**3'},
                 'HYPE_PRIM_AMP':   {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_PHASE': {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_MEAN':  {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_FLAG':  {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'}}
# define the saving varibales
var_encoding_HYPE     = {'HYPE_SPL_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_SPL_COEFF':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_SPL_EXP':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_LIM_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_MIN_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_ZERO_EL':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_AMP':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_PHASE': {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_MEAN':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_FLAG':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9}}
#
new_names_HYPE_values = list(new_names_HYPE.values())
var_info_HYPE_keys = list(var_info_HYPE.keys())
var_encoding_HYPE_keys = list(var_encoding_HYPE.keys())
if not new_names_HYPE_values == var_info_HYPE_keys == var_encoding_HYPE_keys:
    sys.exit('keys and values are not similar')


# network topology
new_names_ntopo = {'ID':        'ID',
                   'ID_next':   'ID_next',
                   'length':    'length',
                   'area':      'area',
                   'slope':     'slope',
                   'is_lake':   'is_lake',
                   'lake_vol':  'lake_vol',
                   'lake_type_Doll': 'lake_type_Doll',
                   'lake_type_Doll_HYPE': 'lake_type_Doll_HYPE',
                   'lake_type_Doll_HYPE_HANASAKI': 'lake_type_Doll_HYPE_HANASAKI'}

var_info_ntopo = {'ID':        {'long_name': 'ID',        'unit': '-'},
                  'ID_next':   {'long_name': 'ID next',   'unit': '-'},
                  'length':    {'long_name': 'length',    'unit': 'm'},
                  'area':      {'long_name': 'area',      'unit': 'm**2'},
                  'slope':     {'long_name': 'slope',     'unit': 'm m**-1'},
                  'is_lake':   {'long_name': 'lake flag',     'unit': '-'},
                  'lake_vol':  {'long_name': 'lake volume',   'unit': 'm**3'},
                  'lake_type_Doll':  {'long_name': 'lake type only Doll', 'unit': '-'},
                  'lake_type_Doll_HYPE':  {'long_name': 'lake type Doll and HYPE', 'unit': '-'},
                  'lake_type_Doll_HYPE_HANASAKI': {'long_name': 'lake type Doll and HYPE and HANASAKI', 'unit': '-'}}

var_encoding_ntopo = {'ID':        {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'ID_next':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'length':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'area':      {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'slope':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'is_lake':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_vol':  {'dtype': 'float64', '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll_HYPE':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll_HYPE_HANASAKI': {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9}}
#
new_names_ntopo_values = list(new_names_ntopo.values())
var_info_ntopo_keys = list(var_info_ntopo.keys())
var_encoding_ntopo_keys = list(var_encoding_ntopo.keys())
if not new_names_ntopo_values == var_info_ntopo_keys == var_encoding_ntopo_keys:
    sys.exit('keys and values are not similar')


##
new_names = {**new_names_HYPE, **new_names_Hanasaki, **new_names_Doll, **new_names_ntopo}
var_info = {**var_info_HYPE, **var_info_Hanasaki, **var_info_Doll, **var_info_ntopo}
var_encoding = {**var_encoding_HYPE, **var_encoding_Hanasaki, **var_encoding_Doll, **var_encoding_ntopo}


print(new_names)
print(var_encoding)
print(var_info)


df.rename(columns=new_names, inplace=True)

temp = list(new_names.values())
df = df[temp] # subset to the existing column names

print(df)

df = df.set_index('ID')

ds = df.to_xarray()

ds.attrs['Conventions'] = 'CF-1.6'
ds.attrs['License']     = 'The data were written by Shervan Gharari. They are under GPL.'
ds.attrs['history']     = 'Created ' + time.ctime(time.time())
ds.attrs['source']      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes'


for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])

if os.path.isfile('../data/network_topology/Network_topology_HDMA_with_lake.nc'):
    os.remove('../data/network_topology/Network_topology_HDMA_with_lake.nc')

ds = ds.rename_dims({'ID': 'n'})
ds.to_netcdf('../data/network_topology/Network_topology_HDMA_with_lake.nc',\
             encoding = var_encoding)

#
ds = xr.open_dataset('../data/network_topology/Network_topology_HDMA_with_lake.nc')

ds


{'EL_SPL': 'HYPE_SPL_EL', 'SPL_COEFF': 'HYPE_SPL_COEFF', 'SPL_EXP': 'HYPE_SPL_EXP', 'EL_LIM': 'HYPE_LIM_EL', 'EL_MIN': 'HYPE_MIN_EL', 'EL_ZERO': 'HYPE_ZERO_EL', 'OPS_AMP': 'HYPE_PRIM_AMP', 'OPS_PHA': 'HYPE_PRIM_PHASE', 'OPS_MEAN': 'HYPE_PRIM_MEAN', 'PRM_FLAG': 'HYPE_PRIM_FLAG', 'H06_alpha': 'H06_alpha', 'H06_envfact': 'H06_envfact', 'H06_c1': 'H06_c1', 'H06_c2': 'H06_c2', 'H06_exponent': 'H06_exponent', 'H06_denominator': 'H06_denominator', 'H06_c_compare': 'H06_c_compare', 'H06_frac_Sdead': 'H06_frac_Sdead', 'H06_I_mem_F': 'H06_I_mem_F', 'H06_D_mem_F': 'H06_D_mem_F', 'H06_I_mem_L': 'H06_I_mem_L', 'H06_D_mem_L': 'H06_D_mem_L', 'H06_E_rel': 'H06_E_rel_init', 'H06_purpose': 'H06_purpose', 'inflow_Jan': 'H06_I_Jan', 'inflow_Feb': 'H06_I_Feb', 'inflow_Mar': 'H06_I_Mar', 'inflow_Apr': 'H06_I_Apr', 'inflow_May': 'H06_I_May', 'inflow_Jun': 'H06_I_Jun', 'inflow_Jul': 'H06_I_Jul', 'inflow_Aug': 'H06_I_Aug', 'inflow_Sep': 'H06_I_Sep', 'inflow_Oct': 'H06_I_Oct', 'inflow_Nov': 'H06_I_Nov', 'inflow

<xarray.Dataset>
Dimensions:                       (n: 290)
Coordinates:
    ID                            (n) float64 ...
Dimensions without coordinates: n
Data variables: (12/61)
    HYPE_SPL_EL                   (n) float64 ...
    HYPE_SPL_COEFF                (n) float64 ...
    HYPE_SPL_EXP                  (n) float64 ...
    HYPE_LIM_EL                   (n) float64 ...
    HYPE_MIN_EL                   (n) float64 ...
    HYPE_ZERO_EL                  (n) float64 ...
    ...                            ...
    slope                         (n) float64 ...
    is_lake                       (n) float64 ...
    lake_vol                      (n) float64 ...
    lake_type_Doll                (n) float64 ...
    lake_type_Doll_HYPE           (n) float64 ...
    lake_type_Doll_HYPE_HANASAKI  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Tue Aug 22 20:21:08 2023
    source:       Written by test script of utilities (https://github.com/She...